# Advent of Code 2022 - Day 7 of 25

## Problem 1:

--- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
Directory d has total size 24933642.
As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?



In [1]:
import re
import logging

logging.basicConfig(
    filename='parse.log', 
    encoding='utf-8', 
    level=logging.ERROR)

with open('data/aoc_2022_day7.txt') as f:
    lines = [line.replace('\n', '') for line in f]
    
lines[:10]

['$ cd /',
 '$ ls',
 'dir fpljqj',
 '171526 ghtzhjwf.nls',
 'dir gsdsbld',
 'dir hbmjtb',
 '296801 mjfjqw.ccv',
 'dir nfn',
 'dir qmrsvfvw',
 '102565 qqjnqb.chd']

In [2]:
# Patterns to match

pattern_cd = re.compile(r'^[$] (cd) (.*)')
pattern_ls = re.compile(r'^[$] (ls)')
pattern_file = re.compile(r'^(\d*) (.*)')
pattern_dir = re.compile(r'^(dir) (.*)')

In [3]:
class Directory():
    
    def __init__(self, name, items):
        self.name = name
        self.items = items
        self.path = self.get_path()
        self.level = self.get_level()
        
    def add_item(self, item):
        self.items.append(item)
        
    def get_path(self):
        return self.name.split('/')
    
    def get_level(self):
        return len(self.name.split('/'))
        
    def print_directory_info(self):
        print(f'''
Name:           {self.name}
Level:          {self.level}
Items:          {self.items}
Path:           {self.get_path()}
''')

In [4]:
def parse_commands(lines):
    '''
    Iterate through each command and keep track of what is going on.
    
    Input:
        + lines - list of commands
    
    Output:
        + directory_dict - dictionary of Directory objects containing
          information derived from command history
    '''

    current_path = []
    directory_dict = {}

    for index, line in enumerate(lines):

        logging.debug(f'\nIndex:{index}\nThe line is: {line}')
        change_dir = re.findall(pattern_cd, line)
        list_files = re.findall(pattern_ls, line)
        file = re.findall(pattern_file, line)
        directory = re.findall(pattern_dir, line)

        if change_dir:

            if change_dir[0][1] == '/':

                logging.debug(f'--Changing directory from {current_path} to HOME')
                current_path = ['home']

            elif change_dir[0][1] == '..':

                logging.debug('--Going up a level')
                current_path = current_path[:-1]

            else:

                logging.debug(f'--Changing directory from {current_path} to {change_dir[0][1]}')
                current_path.append(change_dir[0][1])

            _name = '/'.join(current_path)
            if _name not in directory_dict:
                logging.debug(f'--Encountered previously unseen directory {_name}')
                directory_dict[_name] = Directory(_name, [])
                
        elif list_files:
            logging.debug(f'--Listing files and directories in {current_path}')

        elif file:
            logging.debug(f'--Located file {file[0][1]} in {current_path}')
            directory_dict[_name].add_item((file[0][0], file[0][1]))

        elif directory:

            logging.debug(f'--Located directory {directory[0][1]} in {current_path}')
            directory_dict[_name].add_item(directory[0][1])

        else:
            logging.debug(f'--Unexpected event occured...')

    return directory_dict

In [5]:
def get_total_filesize(directory, directory_dict):
    
    '''
    Loop through all directories.
        For each directory, add the filesizes and recursively retrieve the directory sizes.
        
    Input: 
        + directory - specific directory in directory_dict
        + directory_dict - dictionary of Directory objects
        
    Output:
        + total_filesize int
    '''
    
    total_filesize = 0
    
    for item in directory.items:
        
        if type(item) == tuple:
            total_filesize += int(item[0])

        if type(item) == str:
            subfolder_name = directory.name + '/' + item
            logging.debug(f'Subfolder found {subfolder_name}')
            total_filesize += int(get_total_filesize(directory_dict[subfolder_name], directory_dict))
            
    return total_filesize

In [6]:
def get_filesizes(directory_dict):
    
    '''
    Loop over entire directory_dict to get all filesizes
    
    Input: 
        + directory_dict - dictionary of Directory objects
        
    Output:
        + dictionary of total filesize, same keys as directory_dict
    
    '''
    
    max_level = max([directory_dict[key].level for key in directory_dict.keys()])
    directory_size = {}

    for level in range(max_level, 0, -1):
        for key in directory_dict.keys():
            directory_size[key] = get_total_filesize(directory_dict[key], directory_dict)

    return directory_size

In [7]:
directory_dict = parse_commands(lines)
directory_sizes = get_filesizes(directory_dict)

In [8]:
# Part 1 Answer

sum([values for key, values in directory_sizes.items() if values <= 100000])

1783610

## Problem 2:

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

Delete directory e, which would increase unused space by 584.
Delete directory a, which would increase unused space by 94853.
Delete directory d, which would increase unused space by 24933642.
Delete directory /, which would increase unused space by 48381165.
Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [9]:
FILESYSTEM_SPACE = 70000000
NEEDED_UNUSED = 30000000

FILESYSTEM_SPACE - directory_sizes['home']
NEEDED_UNUSED - (FILESYSTEM_SPACE - directory_sizes['home'])

4359867

In [10]:
min_value = [('home', directory_sizes['home'])]
for key, value in directory_sizes.items():
    
    if value > NEEDED_UNUSED - (FILESYSTEM_SPACE - directory_sizes['home']):
        if value < min_value[-1][1]:
            min_value.append((key,value))
print(f'Smallest value found meeting requirements is {min_value[-1]}')

Smallest value found meeting requirements is ('home/nfn/qmrsvfvw/fpljqj/tdnp', 4370655)
